In [27]:
import os
import csv
import json

import pandas as pd
from sklearn.model_selection import train_test_split

SEED = 42

In [28]:
data_path = '../../data/google_translate.jsonl'
with open(data_path, 'r', encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

In [29]:
data[0]

{'message_tree_id': '001199c7-7135-4e64-bee9-48ea862243b4',
 'tree_state': 'ready_for_export',
 'prompt': {'message_id': '001199c7-7135-4e64-bee9-48ea862243b4',
  'user_id': '42398a03-04b3-4028-9d97-1444708e91cd',
  'created_date': '2023-02-04T18:54:01.048716+00:00',
  'text': 'Do you have any information about the Commodore 64?',
  'role': 'prompter',
  'lang': 'en',
  'review_count': 3,
  'review_result': True,
  'deleted': False,
  'synthetic': False,
  'emojis': {'+1': 8, '_skip_reply': 2},
  'replies': [{'message_id': '314ea243-638d-48d6-9f3d-8edf80f85ec2',
    'parent_id': '001199c7-7135-4e64-bee9-48ea862243b4',
    'user_id': '00bbfff8-0e6a-4e46-93cc-65785d78c8bc',
    'created_date': '2023-02-05T00:38:59.927142+00:00',
    'text': "Of course. The Commodore 64 is a 8-bit home computer that was released by Commodore International in August 1982. It is one of the most successful home computers of all time, with over 17 million units sold. It featured 64 kilobytes of RAM, color gra

In [30]:
def tree_to_conversations(tree, split="train"):
    # get pairs for root prompt and replies
    conversations = message_to_conversations(tree["prompt"], '', split=split)
    return conversations


def message_to_conversations(message, previous_conversation="", split="train"):
    conversations = []
    current_message =  message['role'].upper() + ": " + message['translation']
    current_conversation = previous_conversation + current_message
    
    # check if we have any replies
    if 'replies' in message:
        for reply in message['replies']:
            if "translation" not in reply:
                continue
            # if reply['role'] == 'prompter':
            #     input_text = reply['translation']
            #     continue
            # if 'rank' in reply:
            #     rank = reply['rank']
            # else:
            #     rank += 1
            rank = 0
            
            reply_role = reply['role'].upper()
            reply_message = reply_role + ": " + reply['translation']
            
            # conversation with start at current prompt
            conversations.append((current_message, reply_message, rank, split))
            
            # conversations extended with the current prompt
            conversations.append((current_conversation, reply_message, rank, split))
            
            # conversations ....
            conversations.extend(message_to_conversations(reply, current_conversation, split))
    return conversations

In [32]:
tree_to_conversations(data[1])

[('PROMPTER: Metoda DMAIC v akciji\nProduktna skupina opaža, da stopnja odliva strank (stopnja, pri kateri stranke prenehajo poslovati z vami) narašča. Da bi preprečili napredovanje trenda, se uporabi metodologija Six Sigma DMAIC za prepoznavanje problema in razvoj rešitve.\n\nOpredelitev (»definiraj«): Stopnja odliva strank se je v zadnjih 6 mesecih povečala s 3 % na 7 %.\n\nMerjenje: Ekipa ima veliko informacij o tem, kako potencialne stranke postanejo stranke, ne pa toliko o tem, kaj se zgodi potem, ko nekdo postane stranka. Zato je odločeno analizirati in izmeriti vedenje uporabnikov po nakupu izdelka.\n\nAnaliza (»Analyze«): Po opazovanju vedenja uporabnikov, potem ko postanejo dejanske stranke, ekipa ugotavlja, da se nove stranke težje navadijo na nov uporabniški vmesnik izdelka.\n\nIzboljšati: ekipa se odloči implementirati tok vključevanja novih strank, ki uporabnikom pomaga prepoznati in uporabiti ključne dele izdelka. Pri večjih strankah sodelujte z ekipo za uspeh strank, da 

In [49]:
pairs = tree_to_pairs(data[0])

for pair in pairs:
    prompt, reply, rank, split = pair
    print(f"PROMPT: {prompt}")
    print(f"REPLY: {reply}")
    print(f"RANK: {rank}")
    print("====================================="*4)

PROMPT: Uporabnik: Ali imate kakšne informacije o Commodore 64?
REPLY: Seveda. Commodore 64 je 8-bitni domači računalnik, ki ga je izdal Commodore International avgusta 1982. Je eden najuspešnejših domačih računalnikov vseh časov, z več kot 17 milijoni prodanih enot. Vseboval je 64 kilobajtov RAM-a, barvno grafiko in zvočni čip SID. Široko se je uporabljal za igre, programiranje in domače aplikacije. Nasledil ga je Commodore 128.

Še vedno obstaja uspešna skupnost navdušencev nad Commodore 64 po vsem svetu, ki še naprej uživajo v igranju in programiranju v sistemu. Več informacij o Commodore 64 najdete na različnih spletnih mestih in forumih, posvečenih sistemu.

Če imate kakršna koli posebna vprašanja o strojni opremi, programski opremi ali programskih jezikih Commodore 64, jih vprašajte.
RANK: 0
PROMPT: Uporabnik: Ali imate kakšne informacije o Commodore 64?
REPLY: Seveda, tukaj je kratek povzetek trenutnega članka v Wikipediji:

Commodore 64 (običajne okrajšave: C64, CBM 64, C=64) j

### Split the trees in train, validation and test sets

In [18]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=SEED, shuffle=True)
data_val, data_test = train_test_split(data_test, test_size=0.5, random_state=SEED, shuffle=True)

In [19]:
len(data_train), len(data_val), len(data_test)

(6923, 865, 866)

### Get the pairs from the trees

In [20]:
def generate_pairs(data, split="train"):
    pairs = []
    for tree in data:
        pairs += tree_to_pairs(tree, split)
    df = pd.DataFrame(pairs, columns=["prompt", "reply", "rank", "split"])
    return df

In [21]:
pairs_train = generate_pairs(data_train, "train")
pairs_val = generate_pairs(data_val, "val")
pairs_test = generate_pairs(data_test, "test")

In [22]:
len(pairs_train), len(pairs_val), len(pairs_test)

(20968, 2600, 2610)

In [25]:
pairs_train.head()

,prompt,reply,rank,split
0,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Kaj pa igranje lažnivca? Za to igro so potrebn...,0,train
1,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...","Uno: Je klasična igra za otroke in odrasle, pr...",1,train
2,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Igrate lahko:\n - 8 noro\n - celina\n - Lažnivec,2,train
3,Napišite hvalnico šolskemu taboru v slogu Mela...,Spomnim se vas prijatelji\nIn spet sva se sreč...,0,train
4,Napišite hvalnico šolskemu taboru v slogu Mela...,"Najljubši kamp, ​​krila za naše sanje!\nDaješ ...",1,train


In [12]:
pairs_val

,prompt,reply,rank,split
0,"Prosimo, da vljudno zahtevate ponudbo pri seli...",Dame in gospodje\n\nKmalu se selim iz Nürnberg...,0,train
1,"valProsimo, da vljudno zahtevate ponudbo pri s...",Dame in gospodje\n\nKmalu se selim iz Nürnberg...,0,train
2,"Prosimo, da vljudno zahtevate ponudbo pri seli...",Dame in gospodje\n\n30. aprila 2023 se namerav...,1,train
3,"valProsimo, da vljudno zahtevate ponudbo pri s...",Dame in gospodje\n\n30. aprila 2023 se namerav...,1,train
4,Kakšne barve so lahko človeški lasje?,Človeška bitja imajo lahko naravne ali barvane...,0,train
...,...,...,...,...
5195,"valPoiščite besede esdrújula, ki se začnejo s f","fonetika, fosfor, faksimile, bajka, fotoelektr...",0,train
5196,"Poiščite besede esdrújula, ki se začnejo s f","Besede Esdrújula, ki se začnejo s črko ""f"", vk...",1,train
5197,"valPoiščite besede esdrújula, ki se začnejo s f","Besede Esdrújula, ki se začnejo s črko ""f"", vk...",1,train
5198,"Poiščite besede esdrújula, ki se začnejo s f","To je nekaj besed esdrújula, ki se začnejo s č...",2,train


In [13]:
pairs_test

,prompt,reply,rank,split
0,Vzemimo naključno matriko $n\krat n$ z realnim...,"Ocenjevanje natančne verjetnosti, da imajo vse...",0,train
1,testVzemimo naključno matriko $n\krat n$ z rea...,"Ocenjevanje natančne verjetnosti, da imajo vse...",0,train
2,Vzemimo naključno matriko $n\krat n$ z realnim...,"Verjetnost, da imajo lastne vrednosti naključn...",1,train
3,testVzemimo naključno matriko $n\krat n$ z rea...,"Verjetnost, da imajo lastne vrednosti naključn...",1,train
4,"Recimo, da je samovozeči avto v situaciji, ko ...",Avtonomni avtomobili morajo biti programirani ...,0,train
...,...,...,...,...
5215,"testImam elektronski set bobnov, a vsakič, ko ...","Obstaja več stvari, s katerimi lahko poskusite...",0,train
5216,"Imam elektronski set bobnov, a vsakič, ko igra...",Eden od načinov za zmanjšanje tresljajev vašeg...,1,train
5217,"testImam elektronski set bobnov, a vsakič, ko ...",Eden od načinov za zmanjšanje tresljajev vašeg...,1,train
5218,"Imam elektronski set bobnov, a vsakič, ko igra...",Pod komplet lahko poskusite postaviti izolacij...,2,train


In [14]:
# combine all splits to a single dataframe
pairs = pd.concat([pairs_train, pairs_val, pairs_test], ignore_index=True)
pairs.head()

,prompt,reply,rank,split
0,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Kaj pa igranje lažnivca? Za to igro so potrebn...,0,train
1,"trainV mojo hišo bodo prispeli 3 prijatelji, r...",Kaj pa igranje lažnivca? Za to igro so potrebn...,0,train
2,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...","Uno: Je klasična igra za otroke in odrasle, pr...",1,train
3,"trainV mojo hišo bodo prispeli 3 prijatelji, r...","Uno: Je klasična igra za otroke in odrasle, pr...",1,train
4,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Igrate lahko:\n - 8 noro\n - celina\n - Lažnivec,2,train


In [15]:
pairs.to_csv("../../data/context_reply_pairs.csv", index=False)

In [16]:
df = pd.read_csv("../../data/context_reply_pairs.csv")